## Exercise 1 

In [ ]:
import pandas as pd
import numpy as np

#Sets a random seed for reproducibility
np.random.seed(42)

#Defines dimensions
num_rows = 6
num_cols = 4

#Generates random data
data = np.random.rand(num_rows, num_cols)

#Creates row and column labels
row_labels = [f'Row{i+1}' for i in range(num_rows)]
col_labels = [f'Col{j+1}' for j in range(num_cols)]

#Creates DataFrame
df = pd.DataFrame(data, index=row_labels, columns=col_labels)

#Displays the DataFrame
print("DataFrame:")
print(df)

#Summary of the inffo
print("\nSummary (info):")
print(df.info())

#Gives the shape
print("\nShape of DataFrame:")
print(df.shape)

#Gives the statistics
print("\nDescriptive Statistics:")
print(df.describe())

## Exercise 2

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#Loads the iris dataset
iris = sns.load_dataset('iris')


#List of numerical columns
numerical_cols = iris.select_dtypes(include=['float']).columns

#Create boxplots for each numerical column
plt.figure(figsize=(12, 8))
for i, col in enumerate(numerical_cols, 1):
    plt.subplot(2, 2, i)
    plt.boxplot(iris[col])
    plt.title(f'Boxplot of {col}')
    plt.ylabel(col)

#Adjust layout and display
plt.tight_layout()
plt.show()

## Exercise 3

In [ ]:
import seaborn as sns
import pandas as pd
import plotly.express as px

#Load the Tips dataset
tips = sns.load_dataset('tips')

#Groups the data then sum the tips
grouped = tips.groupby(['sex', 'day', 'time'])['tip'].sum().reset_index()

#Creates the sunburst chart
fig = px.sunburst(
    grouped,
    path=['sex', 'day', 'time'],  #hierarchy: sex → day → time
    values='tip',
    title='Total Tips by Gender, Day, and Time',
    color='tip',
    color_continuous_scale='Blues'
)

#Shows the plot
fig.show()

## Exercise 4

In [ ]:
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

#Loads the Tips dataset
tips = sns.load_dataset('tips')

#Defines the feature and target variable
X = tips[['total_bill']]  
y = tips['tip']           

#Initialise and train the linear regression model
model = LinearRegression()
model.fit(X, y)

#Display model parameters
print(f"Intercept: {model.intercept_:.2f}")
print(f"Coefficient: {model.coef_[0]:.2f}")

#Predict tip for a total bill of $30
sample_bill = np.array([[30]])
predicted_tip = model.predict(sample_bill)
print(f"Predicted tip for a $30 bill: ${predicted_tip[0]:.2f}")

## Exercise 5

In [ ]:
#Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import itertools

#Load Titanic dataset
dataset_path = 'https://raw.githubusercontent.com/Koldim2001/test_api/refs/heads/main/titanic.csv'
df = pd.read_csv(dataset_path)

#Select columns that are needed
features = ['Pclass', 'Age', 'Fare', 'SibSp', 'Parch', 'Sex', 'Embarked']
df = df[['Survived'] + features]

#Drops rows with missing values for training
df.dropna(inplace=True)

#Defines which columns are categorical and numerical
cat_features = ['Sex', 'Embarked']
num_features = ['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']

#Sets up a column transformer that one-hot encodes categorical variables.
#Ignores numbers
#Drops the first value
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(drop='first'), cat_features)  
], remainder='passthrough')  

#Splits dataset into training and test sets
train, test = train_test_split(df, test_size=0.2, random_state=42)
X_train = train.drop(columns='Survived')
y_train = train['Survived']
X_test = test.drop(columns='Survived')
y_test = test['Survived']

#Defines pipeline and trains a decision tree classifier
model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=5, min_samples_split=150, random_state=42))
])
model.fit(X_train, y_train)

#Makes predictions and evaluates the performance
preds = model.predict(X_test)
acc = accuracy_score(y_test, preds)
cm = confusion_matrix(y_test, preds)
print("Updated model accuracy:", acc)
print("\nClassification Report:\n", classification_report(y_test, preds, target_names=['Not Survived', 'Survived']))


#Extracts full list of feature names
ohe_feature_names = model.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(cat_features)
full_feature_names = list(ohe_feature_names) + num_features

#Gets feature importance scores from the tree
importances = model.named_steps['classifier'].feature_importances_
assert len(full_feature_names) == len(importances), "Mismatch between features and importances!"

#Shows the decision tree structure itself
plt.figure(figsize=(20, 10))
plot_tree(model.named_steps['classifier'], 
          feature_names=full_feature_names, 
          class_names=['Not Survived', 'Survived'], 
          filled=True, max_depth=5)
plt.title("Decision Tree Visualisation")
plt.tight_layout()
plt.show()

#Train models with increasing depth to compare training vs validation errors
errors_list = []
for md in range(1, 21):
    temp_model = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', DecisionTreeClassifier(max_depth=md, random_state=42))
    ])
    temp_model.fit(X_train, y_train)
    train_err = 1 - temp_model.score(X_train, y_train)
    test_err = 1 - temp_model.score(X_test, y_test)
    errors_list.append({'Max Depth': md, 'Training Error': train_err, 'Validation Error': test_err})

#Convert error tracking into DataFrame and plot it
errors_df = pd.DataFrame(errors_list)

plt.figure(figsize=(10, 6))
plt.plot(errors_df['Max Depth'], errors_df['Training Error'], marker='o', label='Training Error')
plt.plot(errors_df['Max Depth'], errors_df['Validation Error'], marker='o', label='Validation Error')
plt.title("Training vs Validation Error by Tree Depth")
plt.xlabel("Max Tree Depth")
plt.ylabel("Prediction Error (1 - Accuracy)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


### Shows confusion matrix

In [ ]:
#Defines function and shows visualise confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    ticks = np.arange(len(classes))
    plt.xticks(ticks, classes, rotation=45)
    plt.yticks(ticks, classes)
    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

plot_confusion_matrix(cm, classes=['Not Survived', 'Survived'])

#Saves the trained model to a pickle file
with open('../outputs/models/model_dt_updated.pkl', 'wb') as f:
    pickle.dump(model, f)

### Testing model on a single hypothetical passenger

In [12]:
#A new hypothetical passenger
new_passenger = pd.DataFrame({
    'Pclass': [3],
    'Age': [55],
    'Fare': [7.25],
    'SibSp': [0],
    'Parch': [0],
    'Sex': ['male'],
    'Embarked': ['S']
})
prediction = model.predict(new_passenger)
print(f"\nPrediction for new passenger: {prediction}")

#Pritns different messages for if the passenger lives or dies
print("This person is most likely a survivor." if prediction[0] == 1 else "This person most likely perished.")


Prediction for new passenger: [0]
This person most likely perished.


### Shows bar chart

In [ ]:
#Extracts full list of feature names
ohe_feature_names = model.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(cat_features)
full_feature_names = list(ohe_feature_names) + num_features

#Gets feature importance scores from the tree
importances = model.named_steps['classifier'].feature_importances_
assert len(full_feature_names) == len(importances), "Mismatch between features and importances!"

#Shows feature importance using a bar chart
importance_df = pd.DataFrame({
    'feature': full_feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=importance_df, x='importance', y='feature', palette='viridis')
plt.title("Feature Importance in Decision Tree")
plt.tight_layout()
plt.show()